Code này giả sử bạn có một ngữ pháp CNF được biểu diễn dưới dạng dictionary, trong đó mỗi quy tắc được lưu dưới dạng key là non-terminal bên trái và value là danh sách các tuple đại diện cho phần bên phải của quy tắc.

In [17]:
def cky_parse(words, grammar, start_symbol="S"):
    """
    Thực hiện CKY parsing cho câu 'words' với ngữ pháp 'grammar' ở dạng CNF.
    :param words: List các từ trong câu.
    :param grammar: Dictionary biểu diễn ngữ pháp CNF.
                     Ví dụ: { "S": [("NP", "VP")], "VP": [("V", "NP")], "NP": [("Det", "N")], ... }
    :param start_symbol: Ký hiệu bắt đầu của ngữ pháp (mặc định "S").
    :return: Bảng CKY (list 2D) với mỗi ô chứa tập hợp các non-terminal có thể sinh ra đoạn con tương ứng.
    """
    n = len(words)
    # Tạo bảng n x n, mỗi ô là một tập rỗng
    table = [[set() for j in range(n)] for i in range(n)]
    
    # Bước 1: Điền bảng với các quy tắc A -> a cho từng từ đơn.
    for i, word in enumerate(words):
        # Duyệt qua ngữ pháp để tìm các quy tắc có dạng A -> word
        for lhs, productions in grammar.items():
            for prod in productions:
                if len(prod) == 1 and prod[0] == word:
                    table[i][i].add(lhs)
        # In ra thông tin (debug)
            #print(f"table[{i}][{i}] = {table[i][i]}")
    
    # Bước 2: Điền bảng cho các đoạn con có độ dài > 1.
    # l là độ dài đoạn con từ 2 đến n.
    for l in range(2, n + 1):
        for i in range(n - l + 1):
            j = i + l - 1
            # Chia đoạn con từ i đến j tại các vị trí k từ i đến j-1
            for k in range(i, j):
                # Xét tất cả các quy tắc A -> B C
                for lhs, productions in grammar.items():
                    for prod in productions:
                        if len(prod) == 2:
                            B, C = prod
                            # Nếu B có trong table[i][k] và C có trong table[k+1][j] thì thêm A vào table[i][j]
                            if B in table[i][k] and C in table[k+1][j]:
                                table[i][j].add(lhs)
            # print(f"table[{i}][{j}] = {table[i][j]}")
    
    # Kiểm tra xem ký hiệu bắt đầu có ở ô chứa toàn bộ câu không
    return table, (start_symbol in table[0][n - 1])

# Ví dụ ngữ pháp CNF đơn giản
# Ngữ pháp này là một ví dụ rất đơn giản cho câu "book the flight"
grammar = {
    "S": [("NP", "VP")],
    "VP": [("V", "NP")],
    "NP": [("Det", "N"), ("NP", "PP")],
    "PP": [("P", "NP")],
    "Det": [("the",)],
    "N": [("book",), ("flight",)],
    "V": [("book",)],
    "P": [("through",)]
}

# Ví dụ câu để phân tích
sentence = "book the flight".split()

# Thực hiện CKY parsing
table, accepted = cky_parse(sentence, grammar, start_symbol="S")

print("Bảng CKY:")
for row in table:
    print(row)

if accepted:
    print("Câu được chấp nhận theo ngữ pháp (S xuất hiện trong ô [0][n-1]).")
else:
    print("Câu không được chấp nhận theo ngữ pháp.")



Bảng CKY:
[{'V', 'N'}, set(), {'VP'}]
[set(), {'Det'}, {'NP'}]
[set(), set(), {'N'}]
Câu không được chấp nhận theo ngữ pháp.
